## 设备全局内存和数据传输
在当前的 CUDA 系统中，设备通常是配有自己的动态随机访问内存（称为设备全局内存或全局内存）的硬件卡。例如，NVIDIA Volta V100配备了 16GB 或 32GB 的全局内存。将其称为“全局”内存是为了将其与程序员也可访问的其他类型的设备内存区分开。

对于矢量加法核函数，在调用核函数之前，程序员需要在设备全局内存中分配空间并将数据从主机内存传输到设备全局内存中的已分配空间。同样，在设备执行后，程序员需要将结果数据从设备全局内存传输回主机内存，并释放在设备全局内存中分配的不再需要的空间。

cudaMemcpy函数有四个参数。第一个参数是指向要复制的数据对象目标位置的指针。第二个参数指向源位置。第三个参数指定要复制的字节数。第四个参数指示复制涉及的内存类型：从主机到主机，从主机到设备，从设备到主机以及从设备到设备。

In [1]:
from IPython.core.magic import register_cell_magic
import os
import subprocess

@register_cell_magic
def cuda(line, cell):
    # 1. 把 Cell 里的代码保存为文件
    filename = "cuda_code.cu"
    with open(filename, "w") as f:
        f.write(cell)

    # 2. 编译命令 (自动加上了 -arch=sm_75 修复你的报错)
    # 这里的 -arch=sm_75 是专门针对 Colab T4 显卡的
    compile_cmd = "/usr/local/cuda/bin/nvcc -arch=sm_75 -o cuda_code cuda_code.cu"
    
    # 3. 执行编译
    print(f"🔨 Compiling with: {compile_cmd}")
    result = subprocess.run(compile_cmd, shell=True, capture_output=True, text=True)
    
    if result.returncode != 0:
        print("❌ Compilation Failed!")
        print(result.stderr)
        return

    # 4. 运行编译好的程序
    print("🚀 Running...")
    run_cmd = "./cuda_code"
    run_result = subprocess.run(run_cmd, shell=True, capture_output=True, text=True)
    
    # 5. 输出结果
    print(run_result.stdout)
    if run_result.stderr:
        print("Runtime Errors:", run_result.stderr)

print("✅ 自定义 %%cuda 已激活！现在你可以直接写代码了，无需任何参数。")

✅ 自定义 %%cuda 已激活！现在你可以直接写代码了，无需任何参数。


In [ ]:
%% cuda
void vecAdd(float* A_h, float* B_h, float* C_h, int n){
    int size = n * sizeof(float);
    float *A_d, *B_d, *C_d;

    cudaMalloc((void **) &A_d, size);
    cudaMalloc((void **) &B_d, size);
    cudaMalloc((void **) &C_d, size);

    cudaMemcpy(A_d, A_h, size, cudaMemcpyHostToDevice);
    cudaMemcpy(A_d, A_h, size, cudaMemcpyHostToDevice);

    // Kernel invocation code

    cudaMemcpy(C_h, C_d, size, cudaMemcpyDevicetoHost);

    cudaFree(A_d);
    cudaFree(B_d);
    cudaFree(C_d);
    
}

# 核函数和线程
在CUDA C中，核函数指定在并行阶段由所有线程执行的代码。当程序的主机代码调用核函数时，CUDA运行时系统会启动一个线程网格，该网格组织成两级层次结构。每个网格都组织成一个线程块数组，我们将其简称为块。一个网格中的所有块都是相同大小的；每个块在当前系统上最多可以包含1024个线程。

每个线程块中的总线程数是在调用核函数时由主机代码指定的。同一核函数可以在主机代码的不同部分以不同数量的线程调用。对于给定的线程网格，**块中的线程数**可以在名为blockDim的内建变量中找到。blockDim变量是一个结构，包含三个无符号整数字段（x、y和z）。这些字段帮助程序员将线程组织成一维、二维或三维数组。

CUDA核函数可以访问另外两个内建变量（threadIdx和blockIdx），这些变量允许线程彼此区分，并确定每个线程要处理的数据区域。threadIdx变量为每个线程提供**块内**的唯一坐标。

| Qualifier Keyword | Callable From | Executed On | Exexuted by |
|:---:|:---:|:---:| :---:|
| ___host___ | Host | Host | Caller host thread |
| ___global___ | Host | Device | New grid of device threads |
| ___device___ | Device | Device | Caller device thread |

In [ ]:
%% cuda

// Compute vector sum C = A + B
__global__ void vecAddKernel(float* A, float* B, float* C, int n){
    int i = threadIdx.x + blockDim.x * blockIdx.x;
    if(i < n){
        C[i] = A[i] + B[i];
    }
}

在CUDA核函数中，局部变量对于每个线程都是私有的。也就是说，每个线程都会生成一个i的版本。如果使用10,000个线程启动网格，将会有10,000个版本的 i ，每个线程一个版本。由线程分配给其 i 变量的值对其他线程不可见。循环现在被线程网格替代了。整个网格形成了循环的等效部分。网格中的每个线程对应于原始循环的一次迭代。这有时被称为循环并行性，其中原始顺序代码的迭代由线程并行执行。

addVecKernel函数有一个if (i < n)语句。这是因为并非所有的矢量长度都可以表示为块大小的倍数。例如，假设矢量长度是100。最小的有效线程块维度是32。假设我们选择32作为块大小。将需要启动四个线程块来处理所有100个矢量元素。然而，这四个线程块将有128个线程。我们需要禁用第3个线程块中的最后28个线程，以防它们执行原始程序不期望的工作。由于所有线程都将对它们的i值进行与n的比较，因此所有线程将测试它们的i值是否小于n，其中n是100。通过if (i < n)语句，前100个线程将执行加法，而最后的28个线程将不执行。

# 调用核函数
当主机代码调用核函数时，它通过执行配置参数设置网格和线程块的维度。配置参数位于传统C函数参数之前的“<<<”和“>>>”之间。第一个配置参数给出了网格中的块数，第二个指定了每个块中的线程数。

In [ ]:
// Excute kernel 

int vecAdd(float* A， float* B， float* C， int n){

    vecAddKernel<<<ceil(n/256.0), 256>>>(A_d, B_d, C_d, n);
}
// full code of VectorAdd

void vecAdd(float* A_h, float* B_h, float* C_h, int n){
    int size = n * sizeof(float);
    float *A_d, *B_d, *C_d;

    cudaMalloc((void **) &A_d, size);
    cudaMalloc((void **) &B_d, size);
    cudaMalloc((void **) &C_d, size);

    cudaMemcpy(A_d, A_h, size, cudaMemcpyHostToDevice);
    cudaMemcpy(A_d, A_h, size, cudaMemcpyHostToDevice);

    // Kernel invocation code
    vecAddKernel<<<ceil(n/256.0), 256>>>(A_d, B_d, C_d, n);

    cudaMemcpy(C_h, C_d, size, cudaMemcpyDevicetoHost);

    cudaFree(A_d);
    cudaFree(B_d);
    cudaFree(C_d);
    
}


# 多维网格组织
在CUDA中，网格中的所有线程都执行相同的内核函数，并且它们依赖于坐标，即线程索引，以相互区分并识别要处理的数据的适当部分。这些线程被组织成两级层次结构：一个网格由一个或多个块组成，每个块由一个或多个线程组成。块中的所有线程共享相同的块索引，可以通过 blockIdx（内置）变量访问。每个线程还有一个线程索引，可以通过 threadIdx（内置）变量访问。

通常，一个网格是一个三维（3D）的块数组，每个块是一个三维（3D）的线程数组。在调用内核时，程序需要指定每个维度中网格和块的大小。

假设要处理 $62\times76$的图片，对于一个 $16\times16$ 的块，y 方向需要 4 个块， x 方向需要 5 个块。
垂直（横坐标）
$$ row = blockIdx.y * blockDim.y + threadIdx.y $$
水平（纵坐标）
$$ col = blockIdx.x * blockDim.x + threadIdx.x $$

# 矩阵乘法

In [ ]:
// 使用一个线程计算一个 P 元素的矩阵乘法的 Kernel, 两个矩阵大小均为 Width x Width
__global__ void MatrixMulKernel(float* M, float* N, float* K, int Width){
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    if(row < Width && col < Width){
        float value = 0.0f;
        for(int k = 0; k < Width; ++k){
            value += M[row * Width + k] * N[k * Width + col];
        }
    }
    P[row * Width + col] = value;
}